In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=0025a46725d64f31e56342a6ba9ce41ce963280e9b28836d5e5e84a8be85acd6
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.7 MB/s eta 0:00:00


In [ ]:
import re

from datasets import load_dataset, load_metric
import evaluate
import nltk
import nltk.data
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AdamW, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
rouge_score = evaluate.load("rouge")

In [ ]:
# xsci_train = load_dataset('multi_x_science_sum', split='train')
# xsci_val = load_dataset('multi_x_science_sum', split='validation')
# xsci_test = load_dataset('multi_x_science_sum', split='test')

xsci = load_dataset("multi_x_science_sum")

xsci

Generating train split:   0%|          | 0/30369 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5093 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5066 [00:00<?, ? examples/s]

Dataset multi_x_science_sum downloaded and prepared to /root/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 30369
    })
    test: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5093
    })
    validation: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5066
    })
})

In [ ]:
xsci['train'][0]

{'aid': 'math9912167',
 'mid': '1631980677',
 'abstract': 'Author(s): Kuperberg, Greg; Thurston, Dylan P. | Abstract: We give a purely topological definition of the perturbative quantum invariants of links and 3-manifolds associated with Chern-Simons field theory. Our definition is as close as possible to one given by Kontsevich. We will also establish some basic properties of these invariants, in particular that they are universally finite type with respect to algebraically split surgery and with respect to Torelli surgery. Torelli surgery is a mutual generalization of blink surgery of Garoufalidis and Levine and clasper surgery of Habiro.',
 'related_work': 'Two other generalizations that can be considered are invariants of graphs in 3-manifolds, and invariants associated to other flat connections @cite_16 . We will analyze these in future work. Among other things, there should be a general relation between flat bundles and links in 3-manifolds on the one hand and finite covers and b

In [ ]:
pat = re.compile("@cite_[0-9]+")

pat

re.compile(r'@cite_[0-9]+', re.UNICODE)

In [ ]:
DATASET_NAME = "multi_x_science_sum"
DOC_SEP = " ||||| "
BATCH_SIZE = 64

In [ ]:
def preprocess_dataset(example):
    output = {}
    output["abstracts"] = (
        example["abstract"].split("| Abstract: ")[-1]
        + DOC_SEP
        + DOC_SEP.join([x for x in example["ref_abstract"]["abstract"] if x])
    )
    output["related_work"] = pat.sub("@cite", example["related_work"])
    
    return output

In [ ]:
def preprocess_dataset_batched(example):
    output = {}
    output["abstracts"] = []
    output["related_work"] = []
    
    for abstract, ref_abstract in zip(
        example["abstract"], example["ref_abstract"]
    ):
        output["abstracts"].append(
            abstract.split("| Abstract: ")[-1]
            + DOC_SEP
            + DOC_SEP.join([x for x in ref_abstract["abstract"] if x])
        )
    for related_work in example["related_work"]:
        output["related_work"].append(pat.sub("@cite", related_work))
    
    return output

In [ ]:
dataset_processed = {}
for key in xsci.keys():
    dataset_processed[key] = xsci[key].map(
        # preprocess_dataset,
        preprocess_dataset_batched,
        remove_columns=xsci[key].column_names,
        batched=True,
        batch_size=BATCH_SIZE,
    )

Map:   0%|          | 0/30369 [00:00<?, ? examples/s]

Map:   0%|          | 0/5093 [00:00<?, ? examples/s]

Map:   0%|          | 0/5066 [00:00<?, ? examples/s]

In [ ]:
punkt_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
test_pred_baselines = [
    " ".join(
        [
            " ".join(punkt_tokenizer.tokenize(each)[:2]).strip()
            for each in element["abstracts"].split("|||||")
        ]
    ).strip()
    for element in dataset_processed["test"]
]

In [ ]:
rouge = load_metric("rouge")
scores = rouge.compute(
    predictions=test_pred_baselines,
    references=dataset_processed["test"]["related_work"],
    use_stemmer=True,
)

<ipython-input-17-0b69dd7fcf90>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [ ]:
scores

{'rouge1': AggregateScore(low=Score(precision=0.24515739314476476, recall=0.42323717244839393, fmeasure=0.29050090306695564), mid=Score(precision=0.24766862547679672, recall=0.42645571984038255, fmeasure=0.29242545928988045), high=Score(precision=0.25041790224975796, recall=0.4295746196109397, fmeasure=0.29463091098216926)),
 'rouge2': AggregateScore(low=Score(precision=0.04408219233236484, recall=0.07980649774131757, fmeasure=0.05295349081301365), mid=Score(precision=0.044977334860780295, recall=0.08134878345377156, fmeasure=0.053967176744877804), high=Score(precision=0.04598200792354747, recall=0.0830172600458143, fmeasure=0.05505618768147445)),
 'rougeL': AggregateScore(low=Score(precision=0.12224228882600334, recall=0.21481435405546342, fmeasure=0.14532159301596748), mid=Score(precision=0.12365826541273785, recall=0.21673739462133992, fmeasure=0.14639615023309777), high=Score(precision=0.12495844987403612, recall=0.21860584242649256, fmeasure=0.14746184988049157)),
 'rougeLsum': Ag

In [ ]:
CHECKPOINT = "ratishsp/Centrum"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

In [ ]:
tokenizer.add_tokens(DOC_SEP, special_tokens=True)
model.resize_token_embeddings(len(tokenizer))
docsep_token_id = tokenizer.convert_tokens_to_ids(DOC_SEP)

In [ ]:
dataset_tokenized = {}

dataset_tokenized["test"] = tokenizer(
    dataset_processed["test"]["abstracts"],
    padding=True,
    truncation=True,
    max_length=1024,
    return_tensors="pt",
)

In [ ]:
dataset_tokenized["test"]["global_attention_mask"] = np.array([
    [
        1 if token in (tokenizer.cls_token_id, docsep_token_id) else 0 
        for token in each
    ]
    for each in dataset_tokenized["test"]["input_ids"]
])

In [ ]:
xsci

DatasetDict({
    train: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 30369
    })
    test: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5093
    })
    validation: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5066
    })
})

In [ ]:
import itertools

from typing import List, Tuple

parameters = {'max_length': [64, 128], 'num_beams':[2, 4]}

def generate_model(parameters: dict[str, List[int]]) -> List[Tuple[int]]:
    """Creates pseudo representation of GridSearchCV in order to 
     fine-tune LLM."""
    values = parameters.values()
    combinations = itertools.product(*values)

    return list(combinations)

generate_model(parameters)

[(64, 2), (64, 4), (128, 2), (128, 4)]

In [ ]:
model_grid_search = generate_model(parameters=parameters)

led_output_models = []

# Iterate through the parameter combinations and generate output models
for i in range(len(model_grid_search)):
    max_length, num_beams = model_grid_search[i]

    # Generate output model with current parameter combination
    led_output_model = []
    for j in tqdm(range(0, len(dataset_tokenized["test"]["input_ids"]), BATCH_SIZE)):
        input_ids = dataset_tokenized["test"]["input_ids"][j:j+BATCH_SIZE]
        attention_mask = dataset_tokenized["test"]["attention_mask"][j:j+BATCH_SIZE]
        global_attention_mask = dataset_tokenized["test"]["global_attention_mask"][j:j+BATCH_SIZE]

        led_output_model.append(
            model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                global_attention_mask=global_attention_mask,
                no_repeat_ngram_size=3,
                max_length=max_length,
                num_beams=num_beams,
            )
        )

    # Concatenate all output models into a single tensor
    led_output_model = torch.cat(led_output_model)

    # Append output model to list
    led_output_models.append(led_output_model)

100%|██████████| 80/80 [2:26:42<00:00, 110.03s/it]


In [ ]:
result_models = []

for model in led_output_models:
  test_pred_model = tokenizer.batch_decode(
    model,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
  )

  result_models.append(test_pred_model)


print(f"Total number of models: {len(result_models)}. Type of each model: {type(result_models[0])}.")

Total number of models: 4. Type of each model: <class 'list'>.


In [ ]:
len(xsci['test'][0]['related_work'].split(" "))

149

In [ ]:
model_predictions = [token.strip() for token in model for model in result_models]

AttributeError: ignored

In [ ]:
len(result_models[0][0])

360

In [ ]:
for idx, predictions in enumerate(result_models):

  scores = rouge.compute(
      predictions=predictions,
      references=dataset_processed["test"]["related_work"],
      use_stemmer=True,
  )

  print(f"Rouge scores for Model {idx + 1}: {scores}.")

Rouge scores for Model 1: {'rouge1': AggregateScore(low=Score(precision=0.3535943594609414, recall=0.184947452772402, fmeasure=0.23140147919053997), mid=Score(precision=0.35675396358270384, recall=0.18663523381212738, fmeasure=0.2330816134352135), high=Score(precision=0.3600605701723033, recall=0.18851163908593974, fmeasure=0.2347995174884073)), 'rouge2': AggregateScore(low=Score(precision=0.05200945481519225, recall=0.026165474562661547, fmeasure=0.03313952819295136), mid=Score(precision=0.05349611816994272, recall=0.026973770532208785, fmeasure=0.034052975125961685), high=Score(precision=0.055003581871545124, recall=0.027839056548157964, fmeasure=0.03502698606082491)), 'rougeL': AggregateScore(low=Score(precision=0.2082879556361691, recall=0.1097851207885629, fmeasure=0.13674021488368088), mid=Score(precision=0.2103352441022504, recall=0.11096523063048006, fmeasure=0.1378391194375796), high=Score(precision=0.21233376886860053, recall=0.11209583076433194, fmeasure=0.13890736590121067)

In [ ]:
test_pred_model1 = tokenizer.batch_decode(
    led_output_model1,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

In [ ]:
test_pred_model1 = [each.strip() for each in test_pred_model1]

In [ ]:
test_pred_model1[:2]

In [ ]:
scores = rouge.compute(
    predictions=test_pred_model1,
    references=dataset_processed["test"]["related_work"],
    use_stemmer=True,
)

scores